# Python file to Docker, to ECR, to Lambda 

## Manually Setup Docker Directory

lamdba_fxn_name/
├── Dockerfile
├── app/
│   ├── main.py          # Your main Lambda handler
│   ├── helper.py        # Your helper functions
│   └── requirements.txt # Python dependencies

## Custom Variables

In [60]:
lamdba_fxn_name = "api1_disconnect"

In [61]:
# List of AWS managed policy ARNs
managed_policies = [
    "arn:aws:iam::aws:policy/AmazonAPIGatewayAdministrator",
    "arn:aws:iam::aws:policy/AmazonAPIGatewayInvokeFullAccess",
    "arn:aws:iam::aws:policy/service-role/AmazonAPIGatewayPushToCloudWatchLogs",
    "arn:aws:iam::aws:policy/AmazonAthenaFullAccess",
    "arn:aws:iam::aws:policy/AmazonDynamoDBFullAccess",
    "arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess",
]

## Dependencies

In [62]:
from packages.helpers.helpers import joel_boto

## Functions

In [63]:
def get_dependent_variables(lamdba_fxn_name):
    # Naviagte to the directory i want to build in (same name as lambda)
    path_docker = f"../docker/{lamdba_fxn_name}"

    # Get full IAM role ARN from the name
    role = jb.iam.get_role(RoleName=lamdba_fxn_name)
    role_arn = role["Role"]["Arn"]

    # Used to connect lamba to the correct ECR repo
    repo_uri = jb.get_ecr_repo_uri(lamdba_fxn_name)
    repo_image_uri = repo_uri + ':latest'


    return path_docker, role_arn, repo_uri, repo_image_uri

## Dependent Variables

In [64]:
# Connect to custom AWS class
jb = joel_boto()

running local credentials
✅ Logged in to ECR successfully.
✅ Connected to all clients successfully.


In [65]:
jb.create_lambda_iam_role(lamdba_fxn_name, managed_policies)

✅ Created IAM Role: arn:aws:iam::026090519913:role/api1_disconnect
🔗 Attached policy: arn:aws:iam::aws:policy/AmazonAPIGatewayAdministrator
🔗 Attached policy: arn:aws:iam::aws:policy/AmazonAPIGatewayInvokeFullAccess
🔗 Attached policy: arn:aws:iam::aws:policy/service-role/AmazonAPIGatewayPushToCloudWatchLogs
🔗 Attached policy: arn:aws:iam::aws:policy/AmazonAthenaFullAccess
🔗 Attached policy: arn:aws:iam::aws:policy/AmazonDynamoDBFullAccess
🔗 Attached policy: arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess
🔗 Attached policy: arn:aws:iam::aws:policy/AmazonS3FullAccess
🔗 Attached policy: arn:aws:iam::aws:policy/AWSLambda_FullAccess


In [66]:
# Create ECR Repository (one for each lambda fxn)
jb.create_ecr_repository(lamdba_fxn_name)

Repository created successfully:
026090519913.dkr.ecr.us-east-2.amazonaws.com/api1_disconnect


In [67]:
# Grab Dependent variables
path_docker, role_arn, repo_uri, repo_image_uri = get_dependent_variables(lamdba_fxn_name)

In [68]:
repo_uri

'026090519913.dkr.ecr.us-east-2.amazonaws.com/api1_disconnect'

## Build, Push, and Create Lambda

In [69]:
# Build Docker image and push to ECR
jb.build_and_push_to_ECR(path_docker, repo_uri)

# Create a Lambda fxn using Image in ECR
jb.create_lambda_function(lamdba_fxn_name, role_arn, repo_image_uri)

✅ Lambda Created: 2025-06-10T07:29:15.848+0000


## Update Lambda if exists to the latest image in ECR

In [70]:
# Update if already Exists
jb.update_lambda_function(lamdba_fxn_name, repo_image_uri)

ResourceConflictException: An error occurred (ResourceConflictException) when calling the UpdateFunctionCode operation: The operation cannot be performed at this time. The resource arn:aws:lambda:us-east-2:026090519913:function:api1_disconnect is currently in the following state: 'Pending'. StateReasonCode: 'Creating'